<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**ALGORITMOS** -
Prof: Carmen Pellicer Lostao

# Algoritmo de Bernstein-Vazirani

In [ ]:
# initialisation
import numpy as np
import math, cmath

# Importing standard Qiskit libraries
from qiskit.providers.aer import Aer, AerSimulator
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ, execute
from qiskit.providers.ibmq import least_busy, IBMQBackend
from qiskit.tools.jupyter import *

# basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.visualization import plot_bloch_multivector
from qiskit.visualization import array_to_latex

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()


#### EJERCICIO

Crea las siguientes funciones auxiliares:

- __count_to_digital_values__: una funcion que tome el resultado de cuenas del histograma de salida de una ejecucion de una funcion binaria de n a 1 qubit cuantica y los imprima en el siguiente formato:
```
INPUT------OUTPUT-----count----percent
000 ------ 0 ----- 106 ----- 0.103515625
001 ------ 1 ----- 136 ----- 0.1328125
010 ------ 1 ----- 135 ----- 0.1318359375
011 ------ 0 ----- 141 ----- 0.1376953125
100 ------ 1 ----- 104 ----- 0.1015625
101 ------ 0 ----- 125 ----- 0.1220703125
110 ------ 0 ----- 135 ----- 0.1318359375
111 ------ 1 ----- 142 ----- 0.138671875
shots totales 1024
```
- __get_probabilities__: que tome como entrada un vector de estados y devuelva las probabilidades de todos sus compoentes
- __get__phases__: que tome como entrada un vector de estados y devuelva las fases de todos sus compoentes


In [ ]:
def count_to_digital_values(counts):


def get_probabilities(svector):

    
def get_phases(svector):

    

# ORACULOS

Podemos crear una __funcion de oráculo__ para n qubits que codifique el producto interno binario con una cadena dada $s$:

$$f(\{x_0,x_1,x_2,...\}) \rightarrow 0 \textrm{ or } 1 \textrm{ where } x_n \textrm{ is }0 \textrm{ or } 1$$

$$f_s(x) = s \cdot x \mod 2$$

## Oráculo para una cadena binaria dada

#### EJERCICIO

Dada una cadena $s$ de n bits, construye un circuito con n qubits de entrada y 1 qubit de salida, que compute el resultado de la funcion del oráculo de Berstein-Vazirani para una funcion caracterizada por esta cadena.

Ejecuta el circuito y construye el histograma de resultados

In [ ]:
n = 3 # number of qubits used to represent s
s = '011'   # s2s1s0 the hidden binary string
print('s', s)

#creamos el circuito
oracle_qc = 

# Apply the inner-product oracle


#ponemos las puertas de medida


#lo dibujamos


#Medimos que resultados da este circuito funcion


Si ponemos los n __qubits del input__ en superposicion vemos que es la funcion que realiza la operacion que hemos definado. Estamos ejecutando todas los valores de input posibles a la vez.

#### EJERCICIO

Pon todos los n qubits de la entrada en superposicion y calcula la salida del ciruito.

Para ello ejecuta el circuito, obten el histograma y muestra el resultado de las medidas obtendas con el siguiente formato:

```
INPUT------OUTPUT-----count----percent
000 ------ 0 ----- 129 ----- 0.1259765625
001 ------ 1 ----- 131 ----- 0.1279296875
010 ------ 1 ----- 107 ----- 0.1044921875
011 ------ 0 ----- 122 ----- 0.119140625
100 ------ 0 ----- 127 ----- 0.1240234375
101 ------ 1 ----- 122 ----- 0.119140625
110 ------ 1 ----- 150 ----- 0.146484375
111 ------ 0 ----- 136 ----- 0.1328125
shots totales 1024
```



In [ ]:
n=3
s = '011'   # s2s1s0 the hidden binary string
print('s',s)

#creamos el circuito
oracle_qc = 

#ejecucion todas las entradas posibles de input en paralelo

# Apply the inner-product oracle

#ponemos las puertas de medida


#lo dibujamos



#Medimos que resultados da este circuito funcion


## Oraculo con funcion aleatoria

Con todo el codigo anterior definimos una funcion que de forma aleatoria va a decidir como crea el oraculo

### EJERCICIO

Crea una funcion de oráculo de n qubits que genere una cadena binaria aleatoria s y devuelva un circuito que computa la funcion $s.xmod(2)$ que viene caracterizada por ella 

In [ ]:
#crea una cadena aleatoria s de longitud n bits


In [ ]:
### HACER LA FUNCION

def bv_oracle(n):
    #La funcion crea un objeto de QuantumCircuit y lo devuelve
    #El circuito del oraculo tiene n qubit de entrada y uno de salida -> en total n+1 qubits

    oracle_qc = 

    #crea una cadena aleatoria s de longitud n bits
    s=
    print('Creado BV-oraculo con s',s)
    
    #Crea el oraculo


    oracle_qc.name = 'BV-Oracle' #to show in the display circuit
    return oracle_qc

Lo ejecutamos en superposicion para ver la salida de la funcion del oráculo

In [ ]:
#Creamos un circuito de n qubits+1
n=3
circuit = 

#ponemos los qubits de entrada en superposicion


# Le añadimos el oráculo con funcion aleatoria -> Utilizar el metodo.compose de la clase QuantumCircuit -> recordar el parametro inplace del metodo

#ponemos las puertas de medida


#Mostramos el circuito


backend = Aer.get_backend('aer_simulator')

#mostramos histograma resultado

#mostramos la ejecucion de la funcion en binario


# Ejecucion del Algoritmo de Berstein-Vazirani

## Ejecucion Paso a Paso en el simulador

Vamos a hacer una ejecucion paso a paso viendo el vector de estados. Para ello vamos definir una funcion que nos ayudará a visualizar el vector de estados paso a paso

### EJERCICIO

Construye una funcion que tome como entrada un ciruito cuantico, lo ejecute, obtenga el vector de estados resultado y muestre por pantalla su valor en formato latex, las probabilidades y las fases de cada componente del vector y la visualizacion de plot_bloch_multivector

In [ ]:
def svector_viz(qc):
    #tomamos un circuito sin medidas
    #medimos el estado, hay que salvar el vector de estados del circuito (en lugar de poner puertas de medida)
    
    #ejecutamos en el simulador
    
    #obtenemos el vector de estados resultado
    
    #imprimimos el vector en latex
    
    #imprimimos las probabilidades de cada estado de la base
    
    #imprimimos las pases de cada estado de la base
    
    #visualizamos el plot_bloch_multivector


Ejecutamos paso a paso el algoritmo, que no es más que poner en superposicion los estados de los qubits del registro de entrada y de salida:

- en el registro de entrada, todos los qubits deben de estar en el estado |+>
- en el registro de salida el qubit debe de estar en el estado |->

El oraculo es aleatorio y el algoritmo nos dira en una sola ejecucion si la funcion es constante cuando el registro de entrada quede en estado |00...0>. En caso contrario la funcion es balanceada

In [ ]:
#Creamos un circuito de n qubits+1
n=3
circuit = 

#PASO 1 - ponemos los qubits de entrada en superposicion


#ponemos el qubit de salida

#descomentar para mostrar el circuito y vector de estados en este paso
#display(circuit.draw())
#svector_viz(circuit)

# PASO 2- Le añadimos el oráculo con funcion aleatoria



#PASO 3 - Le añadimos las puertas Hadamard


#PASO 4- ponemos las puertas de medida solo para los qubits de la entrada


#PASO 5- Medimos resultados finales


#mostramos histograma resultado

# Ejecucion en un Simulador con ruido

### EJERCICIO

Ejecuta en un simulador con ruido el circuito con el algoritmo de Bersntein-Vazirani, para ello:

- Creamos el circuito cuántico, que llamamos <b>bv_circuit </b>
- Preparamos un simulador con el ruido de un dispositivo real, el `ibmq_vigo`.
- Transpilamos el circuito y lo ejecutamos en el simulador

In [ ]:
#Creamos un circuito de n qubits+1
n=3
circuit = 

#PASO 1 - ponemos los qubits de entrada en superposicion


#ponemos el qubit de salida


# PASO 2- Le añadimos el oráculo con funcion aleatoria


#PASO 3 - Le añadimos las puertas Hadamard



#PASO 4- ponemos las puertas de medida solo para los qubits de la entrada



#Mostramos el circuito


In [ ]:
from qiskit.providers.fake_provider import FakeVigo

# Tomamos un backend ruidoso ficticio de fake providers


# creamos el simulador


# Transpilamos el circuito para mapearlo a las puertas ruidosass


# Ejecutamos el circuito y obtenemos el resultado

# Ejecucion en un BackEnd cuántico real

### EJERCICIO

Ejecuta en un backend real cuantico el circuito con el algoritmo de Bersntein-Vazirani, para ello:

- Creamos el circuito cuántico, que llamamos <b>bv_circuit </b>
- Lo ejecutamos primero en el simulador para ver si funciona bien antes de enviarlo a un BackEnd cuántico real
- Buscamos el backend que este menos cargado de trabajos
- Transpilamos el circuito y lo enviamos para ejecucion

In [ ]:
#Construimos el circuito

#ejecutamos en simulador para comprobar que esta ok

In [ ]:
# buscamos el backend menos ocupado - using a real qc backend: lest busy


In [ ]:
#lanzamos la ejecucion del job

Y cuando termine la ejecucion, consultaremos el resultado